In [ ]:
##装载加启动
import os
import sys
import binascii
import subprocess
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
w = binascii.unhexlify("73642d7765627569").decode('ascii')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
!git config --global advice.detachedHead false
enable_gpu_check=True
def check_gpu(enable_gpu_check):
    if enable_gpu_check:
        physical_devices = tf.config.list_physical_devices('GPU')
        if len(physical_devices) > 0:
            print("GPU is available")
        else:
            sys.exit("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")
    else:
        print("已跳过GPU检测！")
check_gpu(enable_gpu_check)


use_drive = True
if use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')
else:
    print('Google Drive 未挂载!')

def run_code(code):
    try:
        output = subprocess.check_output(code, stderr=subprocess.STDOUT, shell=True)
        print(output.decode())
    except subprocess.CalledProcessError as e:
        print(f"run code '{e.cmd}' failed with error:")
        print(e.output.decode())

sd_version = "\u6B63\u5E38\u7248 master"
custom_version =""
custom_branch =""
os.chdir('/content')
if sd_version =="\u6B63\u5E38\u7248 master":
        sd_code = f"git clone -b master https://github.com/AUTOMATIC1111/{sdw}"
elif sd_version == "\u7A33\u5B9A\u7248 v1.1.0":
        sd_code = f"git clone -b v1.1.0 https://github.com/AUTOMATIC1111/{sdw}"
elif sd_version == "\u624B\u673A\u7248 anapnoe":
        sd_code = f"git clone -b master https://github.com/anapnoe/{sdw}"
elif sd_version == "\u5B9E\u9A8C\u7248 dev":
        sd_code = f"git clone -b dev https://github.com/AUTOMATIC1111/{sdw}"
else:
        sd_code = f"git clone -b {custom_branch} {custom_version} {webui_dir}"

run_code(sd_code)
run_code('apt-get -y install -qq aria2 git')

if sd_version == "\u81EA\u5B9A\u4E49\u7248":
    print(f"Using custom version '{custom_version}' from branch '{custom_branch}' at path '{webui_dir}'...")
else:
    print(f"Using '{sd_version}' version from '{sdw}' repository...")
inf('\u2714 Done', 'success', '50px')

!mkdir -p {webui_dir}/cache
os.environ['TRANSFORMERS_CACHE']=f"{webui_dir}/cache"

#安装环境&stability-ai
with capture.capture_output() as cap:
  %cd /content
  !mkdir -p {webui_dir}/repositories/stable-diffusion-stability-ai
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /content/ --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  if not os.path.exists(f'{webui_dir}/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p {webui_dir}/libtcmalloc && cp .libs/libtcmalloc*.so* {webui_dir}/libtcmalloc
    %env LD_PRELOAD= {webui_dir}/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
    !rm *.tar.gz Patch && rm -r /content/gperftools
  else:
    %env LD_PRELOAD= {webui_dir}/libtcmalloc/libtcmalloc_minimal.so.4

  !mv -n /content/sd/stablediffusion/* {webui_dir}/repositories/stable-diffusion-stability-ai && rm -rf /content/sd
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  os.chdir('/content')

  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py
  !pip install open-clip-torch==2.20.0 -qq --no-deps
  !pip install fastapi==0.94.0 -qq
#安装依赖 Requirements
run_code('apt --fix-broken install')
run_code('pip install -qq pycloudflared ngrok')
run_code('pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv')


config_link = "https://raw.githubusercontent.com/Van-wise/sd-colab/main/config.json"

if use_drive:
    if not os.path.exists(os.path.join(gwebui_dir, "config.json")):
        !mkdir -p {gwebui_dir}
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {gwebui_dir} -o config.json
        !cp {gwebui_dir}/config.json {webui_dir}/config.json
    else:
        !cp {gwebui_dir}/config.json {webui_dir}/config.json
else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {webui_dir} -o config.json
!git clone https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {webui_dir}/extensions/$sdw-localization-zh_Hans  # hanamizuki-ai 中文
!sed -i.bak 's|"localization": ""|"localization": "zh-Hans (Testing)"|' {webui_dir}/config.json

#必备插件
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {webui_dir}/extensions/$w-infinite-image-browsing # zanllp 图片浏览
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Van-wise/sd-colab/main/.env -d {webui_dir}/extensions/$w-infinite-image-browsing -o .env
!git clone https://github.com/Physton/$w-prompt-all-in-one {webui_dir}/extensions/$w-prompt-all-in-one # 提示词插件
!git clone https://github.com/etherealxx/batchlinks-webui {webui_dir}/extensions/batchlinks-webui # 模型文件批量下载

!git clone https://github.com/camenduru/$w-tunnels {webui_dir}/extensions/$w-tunnels  # 用于 -share -cloudflared
!git clone https://github.com/Mikubill/$w-controlnet {webui_dir}/extensions/$w-controlnet # controlnet
!git clone https://github.com/KohakuBlueleaf/a1111-$w-lycoris {webui_dir}/extensions/a1111-$w-lycoris  # lycoris
!git clone https://github.com/AUTOMATIC1111/$sdw-rembg {webui_dir}/extensions/$sdw-rembg    # 背景移除
!git clone https://github.com/thomasasfk/$w-aspect-ratio-helper {webui_dir}/extensions/$w-aspect-ratio-helper #尺寸锁定

#放大插件
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 {webui_dir}/extensions/multidiffusion-upscaler-for-automatic1111 # Tile VAE 放大
#!git clone https://github.com/pkuliyi2015/$w-stablesr {webui_dir}/extensions/$w-stablesr # stablesr 放大
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {webui_dir}/models/ESRGAN -o 4x-UltraSharp.pth


use_theme = "default"
if use_theme == "catppuccin":
    !git clone https://github.com/catppuccin/$sdw {webui_dir}/extensions/$w-catppuccin-theme # catppuccin主题
elif use_theme == "kitchen":
    !git clone https://github.com/canisminor1990/$w-kitchen-theme {webui_dir}/extensions/$w-kitchen-theme # kitchen主题
elif use_theme == "cozy-nest":
    !git clone https://github.com/Nevysha/Cozy-Nest {webui_dir}/extensions/Cozy-Nest # cozy-nest主题
else:
    print("Use Default Theme")

use_donghua = True
if use_donghua:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {webui_dir}/extensions/deforum-for-automatic1111-webui  # deforum
    #!git clone https://github.com/v8hid/infinite-zoom-automatic1111-webui  {webui_dir}/extensions/infinite-zoom-automatic1111-webui   #

use_pose = True
if use_pose:
    !git clone https://github.com/fkunn1326/openpose-editor {webui_dir}/extensions/openpose-editor
    !git clone https://github.com/jexom/$w-depth-lib {webui_dir}/extensions/$w-depth-lib
    !git clone https://github.com/hnmr293/posex {webui_dir}/extensions/posex
    !git clone https://github.com/nonnonstop/$w-3d-open-pose-editor {webui_dir}/extensions/$w-3d-open-pose-editor


ckpt_downlink = "https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors"
ckpt_filename = "chilloutmix_NiPrunedFp32Fix.safetensors"

#链接 models
ckpt_dir = f"{webui_dir}/models/Stable-diffusion"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt_downlink} -d {ckpt_dir} -o {ckpt_filename}

#链接 Lora
Lora_dir = f"{webui_dir}/models/Lora"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {Lora_dir} -o ShojoVibev_11.safetensors
!git clone https://huggingface.co/embed/lora /content/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/lora/* {webui_dir}/models/Lora && rm -rf /content/lora

#链接 VAE
VAE_dir = f"{webui_dir}/models/VAE"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {VAE_dir} -o vae-ft-mse-840000-ema-pruned.safetensors

#链接 embeddings
embeddings_dir = f"{webui_dir}/embeddings"
!git clone https://huggingface.co/embed/negative /content/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/negative/* {webui_dir}/embeddings && rm -rf /content/negative

#链接 hypernetwork

#链接 scripts
scripts_dir = f"{webui_dir}/scripts"
!wget https://raw.githubusercontent.com/camenduru/$sdw-scripts/main/run_n_times.py -O {webui_dir}/scripts/run_n_times.py

clear_output()
inf('\u2714 Model downloaded, using the trained model.', 'success', '350px')


cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"

controlnet = 1
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

clear_output()
inf('\u2714 Done','success', '50px')


use_drive_model = True
content_way = "\u94FE\u63A5ln"

if use_drive and use_drive_model:
    gckpt_dir = f"{gwebui_dir}/models/Stable-diffusion"
    gLora_dir = f"{gwebui_dir}/models/Lora"
    gVAE_dir = f"{gwebui_dir}/models/VAE"
    gembeddings_dir = f"{gwebui_dir}/embeddings"
    gscripts_dir = f"{gwebui_dir}/scripts"
    gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"
    goutputs_dir = f"{gwebui_dir}/outputs"
    !mkdir -p {gLora_dir} {gVAE_dir} {gembeddings_dir} {gscripts_dir} {gcn_dir} {goutputs_dir} {gckpt_dir}

    if content_way == "\u590D\u5236cp":
        !cp -r {gckpt_dir} {ckpt_dir}
        !cp -r {gLora_dir} {Lora_dir}
        !cp -r {gVAE_dir} {VAE_dir}
        !cp -r {gembeddings_dir} {embeddings_dir}
        !cp -r {gscripts_dir} {scripts_dir}
        !cp -r {gcn_dir} {cn_dir}
        #!cp -r {goutputs_dir} {webui_dir}/outputs
    else:
        ckpt_dir = f"{webui_dir}/models/Stable-diffusion"
        Lora_dir = f"{webui_dir}/models/Lora"
        VAE_dir = f"{webui_dir}/models/VAE"
        scripts_dir = f"{webui_dir}/scripts"
        embeddings_dir = f"{webui_dir}/embeddings"
        cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
        !ln -sf {gckpt_dir} {ckpt_dir}
        !ln -sf {gLora_dir} {Lora_dir}
        !ln -sf {gVAE_dir} {VAE_dir}
        !ln -sf {gembeddings_dir} {embeddings_dir}
        !ln -sf {gscripts_dir} {scripts_dir}
        !ln -sf {gcn_dir} {cn_dir}

import threading
import time
import shutil
from datetime import datetime
from google.colab import files


use_save_config = True

def check_config_changes():
    while True:
        webui_config_path = os.path.join(webui_dir, 'config.json')
        gwebui_config_path = os.path.join(gwebui_dir, 'config.json')
        if not os.path.exists(webui_config_path):
            print('config.json does not exist, waiting...')
            time.sleep(60)
            continue
        webui_config_mtime = os.path.getmtime(webui_config_path)

        try:
            gwebui_config_mtime = os.path.getmtime(gwebui_config_path)
        except FileNotFoundError:
            gwebui_config_mtime = 0

        if webui_config_mtime > gwebui_config_mtime:
            shutil.copy(webui_config_path, gwebui_config_path)
            print(f'config.json updated at {gwebui_config_path}')

        time.sleep(240)
config_thread = threading.Thread(target=check_config_changes)
if use_save_config:
    config_thread.start()


outputs_drive = True
if os.path.exists(f"{webui_dir}/config.json"):
    if use_drive and outputs_drive:
        !sed -i 's|"outdir_txt2img_samples": "outputs/txt2img-images"|"outdir_txt2img_samples": "{gwebui_dir}/outputs/txt2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_samples": "outputs/img2img-images"|"outdir_img2img_samples": "{gwebui_dir}/outputs/img2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_extras_samples": "outputs/extras-images"|"outdir_extras_samples": "{gwebui_dir}/outputs/extras-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_txt2img_grids": "outputs/txt2img-grids"|"outdir_txt2img_grids": "{gwebui_dir}/outputs/txt2img-grids"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_grids": "outputs/img2img-grids"|"outdir_img2img_grids": "{gwebui_dir}/outputs/img2img-grids"|' {webui_dir}/config.json
    else:
        !sed -i 's|"outdir_txt2img_samples": "outputs/txt2img-images"|"outdir_txt2img_samples": "outputs/txt2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_samples": "outputs/img2img-images"|"outdir_img2img_samples": "outputs/img2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_extras_samples": "outputs/extras-images"|"outdir_extras_samples": "utputs/extras-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_txt2img_grids": "outputs/txt2img-grids"|"outdir_txt2img_grids": "outputs/txt2img-grids"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_grids": "outputs/img2img-grids"|"outdir_img2img_grids": "outputs/img2img-grids"|' {webui_dir}/config.json

use_save_outputs = True
auto_download = False
save_count = 0
first_run = True
last_backup_path = None

def zip_outputs(outputs_path, zip_path, save_interval):
    global save_count
    global first_run
    global last_backup_path

    while True:
        time.sleep(save_interval)

        if first_run:
            first_run = False
            continue

        save_count += 1
        backup_name = f"outputs_{datetime.now().strftime('%m_%d')}_{save_count}"
        backup_path = f"{zip_path}/{backup_name}"

        if last_backup_path and os.path.exists(last_backup_path):
            os.remove(last_backup_path)

        try:
            shutil.make_archive(backup_path, 'zip', outputs_path)
            print(f'Backup created at {backup_path}.zip')

            last_backup_path = f"{backup_path}.zip"

            if auto_download:
                files.download(last_backup_path)
        except Exception as e:
            print(f"An error occurred while creating the backup: {e}")


save_interval = 300
outputs_path = f'{gwebui_dir}/outputs' if use_drive and outputs_drive else f'{webui_dir}/outputs'
zip_path = '/content/outputs'
backup_thread = threading.Thread(target=zip_outputs, args=(outputs_path, zip_path, save_interval))
if use_save_outputs:
    backup_thread.start()

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    #!wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

    %cd {webui_dir}
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

    !sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"{webui_dir}/repositories/stable-diffusion-stability-ai\"]@' {webui_dir}/modules/paths.py
    !sed -i 's@\.\.\/@src/@g' {webui_dir}/modules/paths.py
    !sed -i 's@src/generative-models@generative-models@g' {webui_dir}/modules/paths.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --theme dark"

ngrok_auth=""
if ngrok_auth:
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"

!python {webui_dir}/webui.py $wise